In [2]:
import numpy as np
import pandas as pd

MET_MAP = {
    "sleep": 0.9,             
    "sitting": 1.3,           
    "standing": 1.5,          
    "vehicle": 1.3,           
    "walking": 3.3,           
    "bicycling": 8.0,         
    "household-chores": 3.0,  
    "manual-work": 4.0,       
    "sports": 7.0,            
    "mixed-activity": 3.0     
}

def label_to_met(label):
    return MET_MAP[label]

WINDOW_SECONDS = 10
WINDOW_HOURS   = WINDOW_SECONDS / 3600
WINDOW_MINUTES = WINDOW_SECONDS / 60


In [ ]:
def load_labels(csv_path):
    df = pd.read_csv(csv_path)
    return df["true_label"].tolist(), df["pred_label"].tolist()

def convert_to_met(labels):
    return np.array([label_to_met(lbl) for lbl in labels])

def compute_met_errors(true_met, pred_met):
    mae = np.mean(np.abs(pred_met - true_met))
    rmse = np.sqrt(np.mean((pred_met - true_met)**2))
    return mae, rmse

def compute_met_hours(true_met, pred_met):
    true_h = np.sum(true_met * WINDOW_HOURS)
    pred_h = np.sum(pred_met * WINDOW_HOURS)
    diff_h = abs(true_h - pred_h)
    return true_h, pred_h, diff_h

def time_in_zones(met):
    zones = {
        "sedentary": np.sum(met < 1.5),
        "light":     np.sum((met >= 1.5) & (met < 3)),
        "moderate":  np.sum((met >= 3) & (met < 6)),
        "vigorous":  np.sum(met >= 6),
    }
    # convert windows to minutes
    return {z: zones[z] * WINDOW_MINUTES for z in zones}


In [5]:
def run_met_evaluation(csv_path):
    # Load labels
    true_labels, pred_labels = load_labels(csv_path)
    
    # Convert to MET values
    true_met = convert_to_met(true_labels)
    pred_met = convert_to_met(pred_labels)
    
    # Per-window metrics
    mae, rmse = compute_met_errors(true_met, pred_met)

    # Energy expenditure: MET-hours
    true_h, pred_h, diff_h = compute_met_hours(true_met, pred_met)

    # Time in zones
    true_z = time_in_zones(true_met)
    pred_z = time_in_zones(pred_met)

    print("========== MET Evaluation ==========")
    print(f"MAE (MET):   {mae:.4f}")
    print(f"RMSE (MET):  {rmse:.4f}\n")
    print(f"True MET-hours: {true_h:.3f}")
    print(f"Pred MET-hours: {pred_h:.3f}")
    print(f"Difference:     {diff_h:.3f}\n")
    print("Time-in-Zones (minutes):")
    print("  True:", true_z)
    print("  Pred:", pred_z)

    return {
        "mae": mae,
        "rmse": rmse,
        "true_met_hours": true_h,
        "pred_met_hours": pred_h,
        "diff_met_hours": diff_h,
        "true_zones": true_z,
        "pred_zones": pred_z
    }
